# load package and model

In [3]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path

from config_ai.models.text_span_classify import SeqLabelingModel
from config_ai.models.text_span_classify.common import *

from config_ai.evaluate import eval_text_span_classify
from config_ai.data_utils import *
from config_ai.backend import set_tf_config
from config_ai.models import load_model
from config_ai.experiments import get_model_config
from config_ai.utils import read_config
from snippets.utils import *

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
set_tf_config()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['${CONFIG_AI_PATH}/notebooks/text_span_classify',
 '/nfs/pony/chenhao/workspace/python-snippets',
 '${CONFIG_AI_PATH}',
 '/nfs/pony/chenhao/workspace/bert4keras',
 '${CONFIG_AI_PATH}/notebooks/text_span_classify',
 '/nfs/pony/chenhao/envs/config_ai/lib/python38.zip',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8/lib-dynload',
 '',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8/site-packages',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8/site-packages/IPython/extensions',
 '/mnt/raid4/home/chenhao/.ipython']

2021-11-22 14:55:52 [INFO][backend.py:57]:setting tensorflow config...
2021-11-22 14:55:52 [INFO][backend.py:61]:current devices:
2021-11-22 14:55:52 [INFO][backend.py:62]:cpus:[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
2021-11-22 14:55:52 [INFO][backend.py:63]:gpus:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2021-11-22 14:55:52 [INFO][backend.py:64]:setting gpu memory allow growth...
2021-11-22 14:55:52 [INFO][backend.py:67]:setting soft device placement...
2021-11-22 14:55:52 [INFO][backend.py:71]:set tf config done


In [4]:
config_path = "${CONFIG_AI_PATH}/examples/text_span_classify/ner_seq_labeling.ini"

config = read_config(config_path)
config
model_config = get_model_config(config)
# model_config

model = SeqLabelingModel(config=model_config)
model.label2id

2021-11-22 14:55:55 [INFO][utils.py:100]:parsing config with path:${CONFIG_AI_PATH}/examples/text_span_classify/ner_seq_labeling.ini
2021-11-22 14:55:55 [INFO][utils.py:111]:loading base config...
2021-11-22 14:55:55 [INFO][utils.py:100]:parsing config with path:${CONFIG_AI_PATH}/examples/base_config.ini


{'common_config': {'experiment_dir': '/nfs/pony/chenhao/experiment',
  'project_name': 'config_ai_example',
  'is_train': True,
  'is_test': True,
  'is_save': True,
  'save_args': {'format': 'h5', 'tf_serving_version': 1},
  'eval_phase_list': ['train', 'dev'],
  'output_phase_list': ['train', 'dev', 'test'],
  'is_overwrite_experiment': True,
  'default_random_seed': 10,
  'base_config': '${CONFIG_AI_PATH}/examples/base_config.ini',
  'model_cls': 'SeqLabelingModel',
  'model_name': 'seq_labeling_example'},
 'tokenizer_config': {'tokenizer_name': 'bert_word_piece',
  'tokenizer_args': {'vocabs': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt'}},
 'nn_model_config': {'pretrained_model_tag': 'bert',
  'pretrained_model_path': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12'},
 'compile_config': {'optimizer_name': 'adam',
  'optimizer_args': {'learning_rate': 3e-05}},
 'train_config': {'epochs': 5, 'batch_size': 32, 'overwrite_cache': False},
 'test_config': {

2021-11-22 14:55:55 [INFO][core.py:47]:init model with config:
2021-11-22 14:55:55 [INFO][core.py:124]:initializing tokenizer with config:
{
    "tokenizer_name": "bert_word_piece",
    "tokenizer_args": {
        "vocabs": "/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt"
    }
}
2021-11-22 14:55:55 [INFO][core.py:138]:tokenizer initialized with 21128 vocabs


{'O': 0,
 'B_company': 1,
 'I_company': 2,
 'B_name': 3,
 'I_name': 4,
 'B_position': 5,
 'I_position': 6,
 'B_government': 7,
 'I_government': 8,
 'B_organization': 9,
 'I_organization': 10,
 'B_movie': 11,
 'I_movie': 12,
 'B_address': 13,
 'I_address': 14,
 'B_book': 15,
 'I_book': 16,
 'B_scene': 17,
 'I_scene': 18,
 'B_game': 19,
 'I_game': 20}

# process data

In [5]:
train_data_path = config['data_config']['train_data_path']
train_data = model.jload_lines(train_data_path)
f"{train_data_path}: {len(train_data)} items"


eval_data_path = config['data_config']['eval_data_path']
eval_data = model.jload_lines(eval_data_path)
f"{eval_data_path}: {len(eval_data)} items"


test_data_path = config['data_config']['test_data_path']
test_data = model.jload_lines(test_data_path)
f"{test_data_path}: {len(test_data)} items"



train_data[0]

'${CONFIG_AI_PATH}/data/ner/labeled.jsonl: 4 items'

'${CONFIG_AI_PATH}/data/ner/labeled.jsonl: 4 items'

'${CONFIG_AI_PATH}/data/ner/labeled.jsonl: 4 items'

LabeledTextSpanClassifyExample(text='浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，', text_spans=[TextSpan(text='叶老桂', span=(9, 12), label='name', prob=1.0), TextSpan(text='浙商银行', span=(0, 4), label='company', prob=1.0)])

In [6]:
data_manager = DataManager.get_instance(model=model, data=train_data)

In [7]:
features = data_manager.get_features()
feature = next(features)
feature

{'full_text': '浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，',
 'text': '浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，',
 'extra_text': None,
 'token_ids': [101,
  3851,
  1555,
  7213,
  6121,
  821,
  689,
  928,
  6587,
  6956,
  1383,
  5439,
  3424,
  1300,
  1894,
  1156,
  794,
  1369,
  671,
  702,
  6235,
  2428,
  2190,
  758,
  6887,
  7305,
  3546,
  6822,
  6121,
  749,
  6237,
  6438,
  511,
  1383,
  5439,
  3424,
  6371,
  711,
  8024,
  2190,
  4680,
  1184,
  1744,
  1079,
  1555,
  689,
  7213,
  6121,
  5445,
  6241,
  8024,
  102],
 'segment_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['[CLS]',
  '浙',
  '商',
  '银',
  '行',
  '企',
  '业',
  '信',
  '贷',
  '部',
  '叶',
  '老',
  '桂',
  '博',
  '士',
  '则',
  '从',
  '另',
  '一',
  '个',
 

In [8]:
records = data_manager.get_records(mode="train")
next(records)



{'idx': 0,
 'full_text': '浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，',
 'text': '浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，',
 'extra_text': None,
 'token_ids': [101,
  3851,
  1555,
  7213,
  6121,
  821,
  689,
  928,
  6587,
  6956,
  1383,
  5439,
  3424,
  1300,
  1894,
  1156,
  794,
  1369,
  671,
  702,
  6235,
  2428,
  2190,
  758,
  6887,
  7305,
  3546,
  6822,
  6121,
  749,
  6237,
  6438,
  511,
  1383,
  5439,
  3424,
  6371,
  711,
  8024,
  2190,
  4680,
  1184,
  1744,
  1079,
  1555,
  689,
  7213,
  6121,
  5445,
  6241,
  8024,
  102],
 'segment_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['[CLS]',
  '浙',
  '商',
  '银',
  '行',
  '企',
  '业',
  '信',
  '贷',
  '部',
  '叶',
  '老',
  '桂',
  '博',
  '士',
  '则',
  '从',
  '另',
  '一

# build&compile model

In [9]:
nn_model_args = dict(**config["nn_model_config"])
# nn_model_args.update(transformer_kwargs=dict(dropout_rate=0.3))
nn_model_args

model.build_model(**nn_model_args)



{'pretrained_model_tag': 'bert',
 'pretrained_model_path': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12'}

2021-11-22 14:56:01.452370: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-11-22 14:56:01.459187: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3597740000 Hz
2021-11-22 14:56:01.459665: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563827a7f800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-22 14:56:01.459685: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-22 14:56:01.556875: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563827b19d10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-11-22 14:56:01.556904: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2021-11-22 14:56:01.558155: I tensorflow

In [11]:
compile_args = dict(**config["compile_config"])
# compile_args.update(rdrop_alpha=4)
compile_args
model.compile_model(**compile_args)


{'optimizer_name': 'adam', 'optimizer_args': {'learning_rate': 3e-05}}

2021-11-22 14:56:22 [INFO][seq_labeling.py:95]:compile model with optimizer_name:adam, optimizer_args:{'learning_rate': 3e-05}
2021-11-22 14:56:22 [INFO][tf_core.py:126]:number of devices: 1, use SINGLE scope
2021-11-22 14:56:23 [INFO][losses.py:101]:build loss layer with loss function:<function sparse_categorical_crossentropy at 0x7fa6b5e4bd30>
2021-11-22 14:56:23 [INFO][seq_labeling.py:118]:training model's summary:
2021-11-22 14:56:23 [INFO][layer_utils.py:192]:Model: "model_1"
2021-11-22 14:56:23 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-11-22 14:56:23 [INFO][layer_utils.py:190]:Layer (type)                    Output Shape         Param #     Connected to                     
2021-11-22 14:56:23 [INFO][layer_utils.py:195]:==================================================================================================
2021-11-22 14:56:23 [INFO][layer_utils.py:190]:token_ids (InputLayer)       

In [12]:
model.train_model.inputs
model.train_model.outputs
model.train_model.losses
model.train_model.metrics


[<tf.Tensor 'token_ids:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'segment_ids:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'classify_labels:0' shape=(None, None) dtype=int32>]

[<tf.Tensor 'token_classify_model/Identity:0' shape=(None, None, 21) dtype=float32>]

[<tf.Tensor 'loss_layer/Identity:0' shape=() dtype=float32>]

In [13]:
model.get_dataset_info("train")
model.get_dataset_info("test")

({'token_ids': 'int32', 'segment_ids': 'int32', 'classify_labels': 'int32'},
 {'token_ids': (None,), 'segment_ids': (None,), 'classify_labels': (None,)})

({'token_ids': 'int32', 'segment_ids': 'int32'},
 {'token_ids': (None,), 'segment_ids': (None,)})

In [14]:
dataset = data_manager.get_train_dataset(batch_size=1)

item = next(dataset.as_numpy_iterator())
item


{'token_ids': array([[ 101, 3851, 1555, 7213, 6121,  821,  689,  928, 6587, 6956, 1383,
         5439, 3424, 1300, 1894, 1156,  794, 1369,  671,  702, 6235, 2428,
         2190,  758, 6887, 7305, 3546, 6822, 6121,  749, 6237, 6438,  511,
         1383, 5439, 3424, 6371,  711, 8024, 2190, 4680, 1184, 1744, 1079,
         1555,  689, 7213, 6121, 5445, 6241, 8024,  102]], dtype=int32),
 'segment_ids': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32),
 'classify_labels': array([[0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)}

# train_model

In [15]:
train_args = dict(**config["train_config"])
train_args.update(epochs=4,steps_per_epoch=50, batch_size=8, verbose=1)
train_args

model.train(train_data=train_data, **train_args)

{'epochs': 4,
 'batch_size': 8,
 'overwrite_cache': False,
 'steps_per_epoch': 50,
 'verbose': 1}

100%|██████████| 4/4 [00:00<00:00, 4154.83it/s]
2021-11-22 14:56:33 [INFO][tf_core.py:173]:train on 4 tensors


Epoch 1/4


2021-11-22 14:56:38.163391: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


50/50 [==============================] - 6s 115ms/step - loss: 0.7457 - accuracy: 0.8557
Epoch 2/4
50/50 [==============================] - 6s 118ms/step - loss: 0.0202 - accuracy: 1.0000
Epoch 3/4
50/50 [==============================] - 6s 128ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 4/4
50/50 [==============================] - 6s 125ms/step - loss: 0.0046 - accuracy: 1.0000


2021-11-22 14:57:03 [INFO][tf_core.py:189]:training finished


# predict&evaluate

In [16]:
to_pred = train_data[:4]
to_pred

pred = model.predict(to_pred, show_detail=True)
pred[:4]


[LabeledTextSpanClassifyExample(text='浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，', text_spans=[TextSpan(text='叶老桂', span=(9, 12), label='name', prob=1.0), TextSpan(text='浙商银行', span=(0, 4), label='company', prob=1.0)]),
 LabeledTextSpanClassifyExample(text='生生不息CSOL生化狂潮让你填弹狂扫', text_spans=[TextSpan(text='CSOL', span=(4, 8), label='game', prob=1.0)]),
 LabeledTextSpanClassifyExample(text='那不勒斯vs锡耶纳以及桑普vs热那亚之上呢？', text_spans=[TextSpan(text='那不勒斯', span=(0, 4), label='organization', prob=1.0), TextSpan(text='锡耶纳', span=(6, 9), label='organization', prob=1.0), TextSpan(text='桑普', span=(11, 13), label='organization', prob=1.0), TextSpan(text='热那亚', span=(15, 18), label='organization', prob=1.0)]),
 LabeledTextSpanClassifyExample(text='加勒比海盗3：世界尽头》的去年同期成绩死死甩在身后，后者则即将赶超《变形金刚》，', text_spans=[TextSpan(text='加勒比海盗3：世界尽头》', span=(0, 12), label='movie', prob=1.0), TextSpan(text='《变形金刚》', span=(33, 39), label='movie', prob=1.0)])]

100%|██████████| 4/4 [00:00<00:00, 3584.88it/s]
2021-11-22 14:57:06 [INFO][tf_core.py:204]:predicting with tf model...
1it [00:00, 20.78it/s]
2021-11-22 14:57:06 [INFO][decorators.py:28]:function:_model_predict cost:0.050 seconds
2021-11-22 14:57:06 [INFO][seq_labeling.py:155]:tokens:['[CLS]', '浙', '商', '银', '行', '企', '业', '信', '贷', '部', '叶', '老', '桂', '博', '士', '则', '从', '另', '一', '个', '角', '度', '对', '五', '道', '门', '槛', '进', '行', '了', '解', '读', '。', '叶', '老', '桂', '认', '为', '，', '对', '目', '前', '国', '内', '商', '业', '银', '行', '而', '言', '，', '[SEP]']
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:0, token:[CLS], pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.9962774>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:1, token:浙, pred:('B_company', <tf.Tensor: shape=(), dtype=float32, numpy=0.99092907>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:2, token:商, pred:('I_company', <tf.Tensor: shape=(), dtype=float32, numpy=0.9949203>)
2021-11-22 14:57:06 [INFO][seq_labe

2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:5, token:cs, pred:('B_game', <tf.Tensor: shape=(), dtype=float32, numpy=0.9895271>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:6, token:##ol, pred:('I_game', <tf.Tensor: shape=(), dtype=float32, numpy=0.98989516>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:7, token:生, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.99900717>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:8, token:化, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.99906415>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:9, token:狂, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.9990689>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:10, token:潮, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.99911445>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:11, token:让, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.99909186>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:12, token:你, pred

2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:23, token:在, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.9992501>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:24, token:身, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.9992397>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:25, token:后, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.9992513>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:26, token:，, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.9992447>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:27, token:后, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.9992384>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:28, token:者, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.99924743>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:29, token:则, pred:('O', <tf.Tensor: shape=(), dtype=float32, numpy=0.9992563>)
2021-11-22 14:57:06 [INFO][seq_labeling.py:158]:idx:30, token:即, pred:('O', <tf.Te

[[TextSpan(text='浙商银行', span=(0, 4), label='company', prob=0.9909290671348572),
  TextSpan(text='叶老桂', span=(9, 12), label='name', prob=0.9921231865882874)],
 [TextSpan(text='CSOL', span=(4, 8), label='game', prob=0.9895271062850952)],
 [TextSpan(text='那不勒斯', span=(0, 4), label='organization', prob=0.9964015483856201),
  TextSpan(text='锡耶纳', span=(6, 9), label='organization', prob=0.9964510202407837),
  TextSpan(text='桑普', span=(11, 13), label='organization', prob=0.996509850025177),
  TextSpan(text='热那亚', span=(15, 18), label='organization', prob=0.9965096116065979)],
 [TextSpan(text='加勒比海盗3：世界尽头》', span=(0, 12), label='movie', prob=0.9928401708602905),
  TextSpan(text='《变形金刚》', span=(33, 39), label='movie', prob=0.9929230213165283)]]

In [17]:
output_data = get_text_span_classify_output(to_pred, pred)
output_data[:4]

[{'text': '浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，',
  'text_spans': [{'text': '叶老桂',
    'span': (9, 12),
    'label': 'name',
    'prob': 1.0},
   {'text': '浙商银行', 'span': (0, 4), 'label': 'company', 'prob': 1.0}],
  'predict': [TextSpan(text='浙商银行', span=(0, 4), label='company', prob=0.9909290671348572),
   TextSpan(text='叶老桂', span=(9, 12), label='name', prob=0.9921231865882874)],
  'tp_set': {('叶老桂', 'name', (9, 12)), ('浙商银行', 'company', (0, 4))},
  'fp_set': set(),
  'fn_set': set()},
 {'text': '生生不息CSOL生化狂潮让你填弹狂扫',
  'text_spans': [{'text': 'CSOL',
    'span': (4, 8),
    'label': 'game',
    'prob': 1.0}],
  'predict': [TextSpan(text='CSOL', span=(4, 8), label='game', prob=0.9895271062850952)],
  'tp_set': {('CSOL', 'game', (4, 8))},
  'fp_set': set(),
  'fn_set': set()},
 {'text': '那不勒斯vs锡耶纳以及桑普vs热那亚之上呢？',
  'text_spans': [{'text': '那不勒斯',
    'span': (0, 4),
    'label': 'organization',
    'prob': 1.0},
   {'text': '锡耶纳', 'span': (6, 9), 'label': 'organization', '

In [18]:
true_spans = [e.text_spans for e in to_pred]
eval_rs = eval_text_span_classify(true_spans, pred)
eval_rs

{'detail': {'name': {'tp': 1,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  'movie': {'tp': 2,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  'organization': {'tp': 4,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  'game': {'tp': 1,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  'company': {'tp': 1,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0}},
 'micro': {'tp': 9,
  'fp': 0,
  'fn': 0,
  'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0},
 'macro': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}

# save&load

In [19]:
model_path = os.path.join(config["common_config"]["experiment_dir"],
                          config["common_config"]["project_name"],
                          config["common_config"]["model_name"],"model")

save_args = dict(**config["common_config"]["save_args"])
del save_args["tf_serving_version"]

save_args
model_path


model.save(path=model_path, **save_args)




{'format': 'h5'}

'/nfs/pony/chenhao/experiment/config_ai_example/seq_labeling_example/model'

2021-11-22 14:57:24 [INFO][core.py:79]:saving model to /nfs/pony/chenhao/experiment/config_ai_example/seq_labeling_example/model
2021-11-22 14:57:24 [INFO][tf_core.py:55]:saving keras model to path:/nfs/pony/chenhao/experiment/config_ai_example/seq_labeling_example/model/nn_model/nn_model.h5
2021-11-22 14:57:26 [INFO][core.py:144]:save model done


In [20]:
loaded_model = load_model(path=model_path)
loaded_model

2021-11-22 14:57:27 [INFO][core.py:90]:loading model from path:/nfs/pony/chenhao/experiment/config_ai_example/seq_labeling_example/model
2021-11-22 14:57:27 [INFO][core.py:47]:init model with config:
2021-11-22 14:57:27 [INFO][core.py:124]:initializing tokenizer with config:
{
    "tokenizer_name": "bert_word_piece",
    "tokenizer_args": {
        "vocabs": "/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt"
    }
}
2021-11-22 14:57:27 [INFO][core.py:138]:tokenizer initialized with 21128 vocabs
2021-11-22 14:57:27 [INFO][tf_core.py:157]:loading keras model from path:/nfs/pony/chenhao/experiment/config_ai_example/seq_labeling_example/model/nn_model/nn_model.h5 with format:h5
2021-11-22 14:57:30 [INFO][layer_utils.py:192]:Model: "token_classify_model"
2021-11-22 14:57:30 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-11-22 14:57:30 [INFO][layer_utils.py:190]:Layer (type)                   

In [21]:
loaded_pred = loaded_model.predict(data=to_pred)
loaded_pred

100%|██████████| 4/4 [00:00<00:00, 2802.27it/s]
2021-11-22 14:57:31 [INFO][tf_core.py:204]:predicting with tf model...
2021-11-22 14:57:31 [INFO][decorators.py:28]:function:_model_predict cost:0.076 seconds
2021-11-22 14:57:32 [INFO][decorators.py:28]:function:_post_predict cost:0.049 seconds


[[TextSpan(text='浙商银行', span=(0, 4), label='company', prob=0.9909290671348572),
  TextSpan(text='叶老桂', span=(9, 12), label='name', prob=0.9921231865882874)],
 [TextSpan(text='CSOL', span=(4, 8), label='game', prob=0.9895271062850952)],
 [TextSpan(text='那不勒斯', span=(0, 4), label='organization', prob=0.9964015483856201),
  TextSpan(text='锡耶纳', span=(6, 9), label='organization', prob=0.9964510202407837),
  TextSpan(text='桑普', span=(11, 13), label='organization', prob=0.996509850025177),
  TextSpan(text='热那亚', span=(15, 18), label='organization', prob=0.9965096116065979)],
 [TextSpan(text='加勒比海盗3：世界尽头》', span=(0, 12), label='movie', prob=0.9928401708602905),
  TextSpan(text='《变形金刚》', span=(33, 39), label='movie', prob=0.9929230213165283)]]